<a href="https://colab.research.google.com/github/budaoweng98/-/blob/main/work%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#导入库#
import os
import shutil
import random
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets
import torchvision.transforms as transforms
#from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import sampler

to_float = torch.float
to_long = torch.long

if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
print(device)

cpu


In [2]:
import zipfile
file_dir = '/cifar-10.zip'  # 你的压缩包路径
zipFile = zipfile.ZipFile(file_dir)
for file in zipFile.namelist():
    zipFile.extract(file, '/content')  # 解压路径
zipFile.close()

In [6]:
train_data = datasets.ImageFolder('/content/train')
test_data = datasets.ImageFolder('/content/test')

In [7]:
def check_accuracy_part34(loader, model):
  #if loader.dataset.train:
  #  print('Checking accuracy on validation set')
  #else:
  #  print('Checking accuracy on test set')   
  num_correct = 0
  num_samples = 0
  model.eval()  # set model to evaluation mode
  with torch.no_grad():
    for x, y in loader:
      x = x.to(device=device, dtype=to_float)  # move to device, e.g. GPU
      y = y.to(device=device, dtype=to_long).clamp(max=5)
      scores = model(x)
      _, preds = scores.max(1)
      num_correct += (preds == y).sum()
      num_samples += preds.size(0)
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
  return acc

In [11]:
main_dir = os.getcwd()
train_dir = main_dir + "/extract_train"
test_dir = main_dir + "/extract_test"

epochs = 1
total = []

for i in range(1):
    #建立存放数据集的文件夹
    if os.path.exists(train_dir) and os.path.exists(test_dir):
        shutil.rmtree(train_dir)
        shutil.rmtree(test_dir)
        os.mkdir(train_dir)
        os.mkdir(test_dir)
    else:
        os.mkdir(train_dir)
        os.mkdir(test_dir)
        
    #生成6个随机数，用于提取数据集中的6类
    class_6 = torch.randperm(10)[:6]
    print("Extract classes: ", class_6, "\n")

    #提取6类并保存至新的文件夹
    for i in range(50000):
        train_label = train_data.imgs[i][1]

        if train_label in class_6:
            train_label_dir = train_dir + "/" + str(train_label)
            if os.path.exists(train_label_dir):
                pass
            else:
                os.mkdir(train_label_dir)
            image_path = train_data.imgs[i][0]
            image = Image.open(image_path)
            save_path = train_label_dir + "/" + str(i) + ".jpg"
            image.save(save_path)

    for i in range(10000):
        test_label = test_data.imgs[i][1]

        if test_label in class_6:
            test_label_dir = test_dir + "/" + str(test_label)
            if os.path.exists(test_label_dir):
                pass
            else:
                os.mkdir(test_label_dir)
            image_path = test_data.imgs[i][0]
            image = Image.open(image_path)
            save_path = test_label_dir + "/" + str(i) + ".jpg"
            image.save(save_path)
            
    #定义预处理操作
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
        ])

    #数据集预处理
    train_data_ = datasets.ImageFolder(train_dir, transform=transform)
    test_data_ = datasets.ImageFolder(test_dir, transform=transform)

    #数据集采样、打包
    n_train = len(train_data_)  #30000

    split = 23250
    indices = list(range(n_train))
    random.shuffle(indices)

    train_loader = torch.utils.data.DataLoader(train_data_, batch_size=64, 
                                              sampler=sampler.SubsetRandomSampler(indices[:split]))
    valid_loader = torch.utils.data.DataLoader(train_data_, batch_size=64, 
                                            sampler=sampler.SubsetRandomSampler(indices[split:]))
    test_loader = torch.utils.data.DataLoader(test_data_, batch_size=64, shuffle=True)
    
    #定义网络与参数更新方法
    model = Net()
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    
    #训练网络
    num_prints = epochs * len(train_loader) // 100 + 1
    acc_history = torch.zeros(num_prints, dtype=to_float)
    iter_history = torch.zeros(num_prints, dtype=to_long)
    model = model.to(device=device)# move the model parameters to CPU/GPU
    for epoch in range(epochs):
        for t, (x, y) in enumerate(train_loader):
            model.train()
            # Move the data to the proper device (GPU or CPU)
            x = x.to(device=device, dtype=to_float)
            y = y.to(device=device, dtype=to_long).clamp(max=5)

            # Forward pass: compute scores and loss
            scores = model(x)
            loss = F.cross_entropy(scores, y)

            optimizer.zero_grad()

            #Backwards pass: compute the gradient
            loss.backward()

            #Update the parameters
            optimizer.step()

            tt = t + epoch * len(train_loader)

            if tt % 100 == 0 or (epoch == epochs - 1 and t == len(train_loader) - 1):
                print("Epoch: %d, Iteration: %d, loss = %.4f" %(epoch, tt, loss.item()))
                acc = check_accuracy_part34(valid_loader, model)
                acc_history[tt // 100] = acc
                iter_history[tt // 100] = tt
                print()
    total.append(acc_history[-1])
    
final_acc = sum(total) / len(total) * 100
print("Final accuracy: %.2f" %final_acc)

Extract classes:  tensor([3, 4, 8, 5, 7, 1]) 

Epoch: 0, Iteration: 0, loss = 1.8019
Got 1071 / 6750 correct (15.87)

Epoch: 0, Iteration: 100, loss = 1.7600
Got 1822 / 6750 correct (26.99)

Epoch: 0, Iteration: 200, loss = 1.7417
Got 2172 / 6750 correct (32.18)

Epoch: 0, Iteration: 300, loss = 1.6832
Got 2495 / 6750 correct (36.96)

Epoch: 0, Iteration: 363, loss = 1.5070
Got 2857 / 6750 correct (42.33)

Final accuracy: 42.33


In [4]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.net = nn.Sequential(
            nn.Conv2d(3, 6, 5),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(6, 16, 5),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(16, 32, 3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            nn.Flatten(),
            nn.Linear(1*1*32, 16),
            nn.ReLU(),
            
            nn.Linear(16, 6),
        )
        
    def forward(self, x):
        return self.net(x)
model = Net()
print(model)
optimizer = optim.SGD(model.parameters(), lr=0.01)

Net(
  (net): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
    (9): ReLU()
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Flatten(start_dim=1, end_dim=-1)
    (12): Linear(in_features=32, out_features=16, bias=True)
    (13): ReLU()
    (14): Linear(in_features=16, out_features=6, bias=True)
  )
)


In [12]:
#定义网络与参数更新方法
model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.01)

epochs = 10
num_prints = epochs * len(train_loader) // 100 + 1
acc_history = torch.zeros(num_prints, dtype=to_float)
iter_history = torch.zeros(num_prints, dtype=to_long)
model = model.to(device=device)# move the model parameters to CPU/GPU
for epoch in range(epochs):
    for t, (x, y) in enumerate(train_loader):
        model.train()
        # Move the data to the proper device (GPU or CPU)
        x = x.to(device=device, dtype=to_float)
        y = y.to(device=device, dtype=to_long).clamp(max=5)
        
        # Forward pass: compute scores and loss
        scores = model(x)
        loss = F.cross_entropy(scores, y)
        
        optimizer.zero_grad()
        
        #Backwards pass: compute the gradient
        loss.backward()
        
        #Update the parameters
        optimizer.step()
        
        tt = t + epoch * len(train_loader)
        
        if tt % 100 == 0 or (epoch == epochs - 1 and t == len(train_loader) - 1):
            print("Epoch: %d, Iteration: %d, loss = %.4f" %(epoch, tt, loss.item()))
            acc = check_accuracy_part34(train_loader, model)
            acc_history[tt // 100] = acc
            iter_history[tt // 100] = tt
            print()

Epoch: 0, Iteration: 0, loss = 1.8135
Got 3855 / 23250 correct (16.58)

Epoch: 0, Iteration: 100, loss = 1.7642
Got 5414 / 23250 correct (23.29)

Epoch: 0, Iteration: 200, loss = 1.7131
Got 7011 / 23250 correct (30.15)

Epoch: 0, Iteration: 300, loss = 1.6113
Got 7689 / 23250 correct (33.07)

Epoch: 1, Iteration: 400, loss = 1.3856
Got 9334 / 23250 correct (40.15)

Epoch: 1, Iteration: 500, loss = 1.2405
Got 10537 / 23250 correct (45.32)

Epoch: 1, Iteration: 600, loss = 1.2700
Got 10640 / 23250 correct (45.76)

Epoch: 1, Iteration: 700, loss = 1.2182
Got 10642 / 23250 correct (45.77)

Epoch: 2, Iteration: 800, loss = 1.1614
Got 11425 / 23250 correct (49.14)

Epoch: 2, Iteration: 900, loss = 1.1742
Got 11922 / 23250 correct (51.28)

Epoch: 2, Iteration: 1000, loss = 1.3511
Got 12282 / 23250 correct (52.83)

Epoch: 3, Iteration: 1100, loss = 1.1026
Got 12249 / 23250 correct (52.68)

Epoch: 3, Iteration: 1200, loss = 1.0681
Got 12749 / 23250 correct (54.83)

Epoch: 3, Iteration: 1300, lo

In [17]:
main_dir=os.getcwd()
train_dir=main_dir+"/extract_train"
test_dir=main_dir+"/extract_test"

epochs=1
total=[]

for i in range(5):
    # 建立存放数据集的文件夹
    if os.path.exists(train_dir) and os.path.exists(test_dir):
        shutil.rmtree(train_dir)
        shutil.rmtree(test_dir)
        os.mkdir(train_dir)
        os.mkdir(test_dir)
    else:
        os.mkdir(train_dir)
        os.mkdir(test_dir)

    # 生成6个随机数，用于提取数据集中的6类
    class_6=torch.randperm(10)[:6]
    print("Extract classes: ", class_6, "\n")

    # 提取6类并保存至新的文件夹
    for i in range(50000):
        train_label=train_data.imgs[i][1]

        if train_label in class_6:
            train_label_dir=train_dir+"/"+str(train_label)
            if os.path.exists(train_label_dir):
                pass
            else:
                os.mkdir(train_label_dir)
            image_path=train_data.imgs[i][0]
            image=Image.open(image_path)
            save_path=train_label_dir+"/"+str(i)+".jpg"
            image.save(save_path)

    for i in range(10000):
        test_label=test_data.imgs[i][1]

        if test_label in class_6:
            test_label_dir=test_dir+"/"+str(test_label)
            if os.path.exists(test_label_dir):
                pass
            else:
                os.mkdir(test_label_dir)
            image_path=test_data.imgs[i][0]
            image=Image.open(image_path)
            save_path=test_label_dir+"/"+str(i)+".jpg"
            image.save(save_path)

    # 定义预处理操作
    Transforms = transforms.Compose([
      transforms.Resize((224, 224)),
      transforms.RandomHorizontalFlip(p=0.5),
      transforms.ToTensor(),
      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

    # 数据集预处理
    train_data_=datasets.ImageFolder(train_dir, transform=transform)
    test_data_=datasets.ImageFolder(test_dir, transform=transform)

    # 数据集采样、打包
    n_train=len(train_data_)  # 30000

    split=23250
    indices=list(range(n_train))
    random.shuffle(indices)
    test_loader=torch.utils.data.DataLoader(test_data_, batch_size=6000, shuffle=True)

    # 定义网络与参更新方法
    model=Net()
    optimizer = optim.SGD(model.parameters(), lr=0.01)

    # 训练网络
    num_prints=epochs*len(test_loader)//100+1
    acc_history=torch.zeros(num_prints, dtype=to_float)
    iter_history=torch.zeros(num_prints, dtype=to_long)
    model=model.to(device=device)  # move the model parameters to CPU/GPU
    for epoch in range(epochs):
        for t, (x, y) in enumerate(test_loader):
            model.eval()
            # Move the data to the proper device (GPU or CPU)
            x=x.to(device=device, dtype=to_float)
            y=y.to(device=device, dtype=to_long).clamp(max=5)
            tt=t+epoch*len(test_loader)

            if tt%100==0 or (epoch==epochs-1 and t==len(test_loader)-1):
                print("Epoch: %d, Iteration: %d, loss = %.4f"%(epoch, tt, loss.item()))
                acc=check_accuracy_part34(test_loader, model)
                acc_history[tt//100]=acc
                iter_history[tt//100]=tt
                print()
    total.append(acc_history[-1])

final_acc=sum(total)/len(total)*100
print("Final accuracy: %.2f"%final_acc)

Extract classes:  tensor([7, 2, 8, 0, 1, 9]) 

Epoch: 0, Iteration: 0, loss = 1.6158
Got 1000 / 6000 correct (16.67)

Extract classes:  tensor([7, 4, 3, 6, 1, 9]) 

Epoch: 0, Iteration: 0, loss = 1.6158
Got 1000 / 6000 correct (16.67)

Extract classes:  tensor([0, 9, 4, 2, 6, 3]) 

Epoch: 0, Iteration: 0, loss = 1.6158
Got 1000 / 6000 correct (16.67)

Extract classes:  tensor([1, 9, 0, 4, 6, 2]) 

Epoch: 0, Iteration: 0, loss = 1.6158
Got 1000 / 6000 correct (16.67)

Extract classes:  tensor([7, 6, 0, 4, 2, 9]) 

Epoch: 0, Iteration: 0, loss = 1.6158
Got 992 / 6000 correct (16.53)

Final accuracy: 16.64
